<a href="https://colab.research.google.com/github/castor-git/castor/blob/master/docs/assistant-quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install the Python SDK and plugin

In [1]:
!pip install pinecone pinecone-plugin-assistant

# 2. Connect to Pinecone

You need an API key to make calls to your assistant.

Use the widget below to generate a key. If you don't have a Pinecone account, the widget will sign you up for the free Starter plan.

In [2]:
# install dependencies

!pip install -qU pinecone-notebooks

from pinecone_notebooks.colab import Authenticate

Authenticate()


Save your API key to use throughout the notebook:

In [3]:
import os
api_key = os.environ.get("pcsk_5S4exG_R1dzxgaX4c4qSac2NXrn1CMBpjeKA33kVT76sToqkSZhU9pqBWbyj4BCvJkTWQF")

# 3. Create assistant

In [4]:
from pinecone import Pinecone

pc = Pinecone(api_key=api_key)

assistant = pc.assistant.create_assistant(
    assistant_name="quickstart-assistant",
    instructions="Use American English for spelling and grammar.", # Description or directive for the assistant to apply to all responses.
    timeout=30 # Maximum seconds to wait for assistant status to become "Ready" before timing out.
)

PineconeConfigurationError: You haven't specified an API key. Please either set the PINECONE_API_KEY environment variable or pass the 'api_key' keyword argument to the Pinecone client constructor.

# 3. Upload data

For this quickstart, [download a sample 10-k filing file](https://s22.q4cdn.com/959853165/files/doc_financials/2023/ar/Netflix-10-K-01262024.pdf) to your local device. Then, upload it to the notebook's local files.

In [ ]:
# Get the assistant

from pinecone import Pinecone

pc = Pinecone(api_key)

assistant = pc.assistant.Assistant(
    assistant_name="quickstart-assistant",
)

# Upload a file.
response = assistant.upload_file(
    file_path="/Netflix-10-K-01262024.pdf",
    metadata={"medium": "filing"},
    timeout=None
)

print("File uploaded.")


# 4. View files

In [ ]:
from pinecone import Pinecone
import pprint

pc = Pinecone(api_key)

assistant = pc.assistant.Assistant(
    assistant_name="quickstart-assistant",
)

# List files in your assistant.
files = assistant.list_files()

pprint.pp(files)

# 5. Chat with assistant

In [ ]:
# Chat with the assistant.
from pinecone_plugins.assistant.models.chat import Message

chat_context = [Message(role="user", content="Who is Netflix's CEO and CFO?")]
response = assistant.chat(messages=chat_context)

pprint.pp(response)

# 6. Evaluate the answer

Evaluate the correctness and completeness of a response from an assistant. For more information, refer to [Understanding evaluation](https://docs.pinecone.io/guides/assistant/understanding-evaluation).

In [ ]:
import requests

answer = response.message.content

eval_data = {
    "question": "Who is Netflix's CEO and CFO?",
    "answer": answer,
    "ground_truth_answer": "Ted Sarandos is co-CEO, Greg Peters is co-CEO, and Spencer Neumann is CFO."
}


headers = {
    "Api-Key": api_key,
    "Content-Type": "application/json"
}

url = "https://prod-1-data.ke.pinecone.io/assistant/evaluation/metrics/alignment"

eval_response = requests.request("POST", url, json=eval_data, headers=headers)

pprint.pp(eval_response.text)

# 7. Retrieve context snippets

You can retrieve the context snippets that Pinecone Assistant uses to generate its responses. This data includes relevant chunks, relevancy scores, and references. For more information, refer to [Understanding context snippets](https://docs.pinecone.io/guides/assistant/understanding-context-snippets).

In [ ]:
response_snippets = assistant.context(query=chat_context[0].content)

for snippet in response_snippets.snippets:
  pprint.pp(snippet)

# 8. Clean up

When you no longer need the quickstart-assistant, delete the it:

In [ ]:
from pinecone import Pinecone
pc = Pinecone(api_key="YOUR_API_KEY")

pc.assistant.delete_assistant(
    assistant_name="example-assistant",
)

# Deleting an assistant also deletes all files uploaded to the assistant.